In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

##Using Convolutions with Complex Images

In the previous labs you used the Fashion MNIST dataset to train an image classifier. In this case you had images that were 28x28 where the subject was centered. In this lab you'll take this to the next level, training to recognize features in an image where the subject can be *anywhere* in the image!

You'll do this by building a horses-or-humans classifier that will tell you if a given image contains a horse or a human, where the network is trained to recognize features that determine which is which.


In the case of Fashion MNIST, the data was built into TensorFlow via Keras. In this case the data isn't so you'll have to do some processing of it before you can train.

First, let's download the data:



In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2024-12-01 12:53:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 142.251.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-12-01 12:53:23 ERROR 404: Not Found.

--2024-12-01 12:53:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 142.251.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-12-01 12:53:23 ERROR 404: Not Found.



The following python code will use the OS library to use Operating System libraries, giving you access to the file system, and the zipfile library allowing you to unzip the data.

In [3]:
import os
import zipfile

# Paths to the ZIP files
horse_or_human_zip = '/tmp/horse-or-human.zip'
validation_horse_or_human_zip = '/tmp/validation-horse-or-human.zip'

# Function to safely extract a ZIP file
def extract_zip(file_path, extract_to):
    if os.path.exists(file_path):
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
                print(f"Extracted {file_path} to {extract_to}")
        except zipfile.BadZipFile:
            print(f"Error: {file_path} is not a valid ZIP file.")
    else:
        print(f"Error: {file_path} does not exist.")

# Extract the ZIP files
extract_zip(horse_or_human_zip, '/tmp/horse-or-human')
extract_zip(validation_horse_or_human_zip, '/tmp/validation-horse-or-human')


Error: /tmp/horse-or-human.zip is not a valid ZIP file.
Error: /tmp/validation-horse-or-human.zip is not a valid ZIP file.


The contents of the .zip are extracted to the base directory `/tmp/horse-or-human`, which in turn each contain `horses` and `humans` subdirectories.

In short: The training set is the data that is used to tell the neural network model that 'this is what a horse looks like', 'this is what a human looks like' etc.

One thing to pay attention to in this sample: We do not explicitly label the images as horses or humans. If you remember with the fashion example earlier, we had labelled 'this is a 1', 'this is a 7' etc.

Later you'll see something called an *ImageGenerator* being used -- and this is coded to read images from subdirectories, and automatically label them from the name of that subdirectory. So, for example, you will have a 'training' directory containing a 'horses' directory and a 'humans' one. ImageGenerator will label the images appropriately for you, reducing a coding step.

Let's define each of these directories:

In [4]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

Now, let's see what the filenames look like in the `horses` and `humans` training directories:

In [6]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to the ZIP files and their extraction directories
horse_or_human_zip = '/tmp/horse-or-human.zip'
validation_horse_or_human_zip = '/tmp/validation-horse-or-human.zip'
train_dir = '/tmp/horse-or-human/'
validation_dir = '/tmp/validation-horse-or-human/'

# Ensure the ZIP files exist (or download if necessary)
if not os.path.exists(horse_or_human_zip) or not os.path.exists(validation_horse_or_human_zip):
    print("One or both ZIP files are missing. Please ensure the files are available.")
else:
    # Extract the ZIP files
    def extract_zip(file_path, extract_to):
        if os.path.exists(file_path):
            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)
                    print(f"Extracted {file_path} to {extract_to}")
            except zipfile.BadZipFile:
                print(f"Error: {file_path} is not a valid ZIP file.")
        else:
            print(f"Error: {file_path} does not exist.")

    extract_zip(horse_or_human_zip, train_dir)
    extract_zip(validation_horse_or_human_zip, validation_dir)

# Check if the directories exist after extraction
if not os.path.exists(train_dir) or not os.path.exists(validation_dir):
    print("Extraction failed. Ensure the ZIP files contain the correct directories.")
else:
    # Step 2: Create Data Generators
    # All images will be rescaled by 1./255
    train_datagen = ImageDataGenerator(rescale=1/255)
    validation_datagen = ImageDataGenerator(rescale=1/255)

    # Flow training images in batches using train_datagen generator
    train_generator = train_datagen.flow_from_directory(
            train_dir,                 # Source directory for training images
            target_size=(300, 300),    # Resize images to 300x300
            batch_size=128,
            class_mode='binary')       # Binary labels for binary_crossentropy

    # Flow validation images in batches using validation_datagen generator
    validation_generator = validation_datagen.flow_from_directory(
            validation_dir,            # Source directory for validation images
            target_size=(300, 300),    # Resize images to 300x300
            batch_size=32,
            class_mode='binary')       # Binary labels for binary_crossentropy


Error: /tmp/horse-or-human.zip is not a valid ZIP file.
Error: /tmp/validation-horse-or-human.zip is not a valid ZIP file.
Extraction failed. Ensure the ZIP files contain the correct directories.


Let's find out the total number of horse and human images in the directories:

In [8]:
import os

# Base training directory
train_dir = '/tmp/horse-or-human'

# Define subdirectories for horse and human images
train_horse_dir = os.path.join(train_dir, 'horses')
train_human_dir = os.path.join(train_dir, 'humans')

# Check if the directories exist before listing files
if os.path.exists(train_horse_dir) and os.path.exists(train_human_dir):
    print('Total training horse images:', len(os.listdir(train_horse_dir)))
    print('Total training human images:', len(os.listdir(train_human_dir)))
else:
    print("Error: One or both of the directories do not exist.")
    print("Check if the dataset has been downloaded and extracted correctly.")


Error: One or both of the directories do not exist.
Check if the dataset has been downloaded and extracted correctly.


Now let's take a look at a few pictures to get a better sense of what they look like. First, configure the matplot parameters:

In [9]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

Now, display a batch of 8 horse and 8 human pictures. You can rerun the cell to see a fresh batch each time:

In [11]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define the directories
train_dir = '/tmp/horse-or-human'
train_horse_dir = os.path.join(train_dir, 'horses')
train_human_dir = os.path.join(train_dir, 'humans')

# Verify the directories exist
if not os.path.exists(train_horse_dir) or not os.path.exists(train_human_dir):
    print("Error: One or both of the directories do not exist.")
else:
    # List filenames in horse and human directories
    train_horse_names = os.listdir(train_horse_dir)
    train_human_names = os.listdir(train_human_dir)

    # Set up parameters for display
    nrows = 4
    ncols = 4
    pic_index = 0  # Start at the first image

    # Set up matplotlib fig, and size it to fit 4x4 pics
    fig = plt.gcf()
    fig.set_size_inches(ncols * 4, nrows * 4)

    # Select next 8 images from each category
    pic_index += 8
    next_horse_pix = [os.path.join(train_horse_dir, fname)
                      for fname in train_horse_names[pic_index-8:pic_index]]
    next_human_pix = [os.path.join(train_human_dir, fname)
                      for fname in train_human_names[pic_index-8:pic_index]]

    # Display images
    for i, img_path in enumerate(next_horse_pix + next_human_pix):
        # Set up subplot; subplot indices start at 1
        sp = plt.subplot(nrows, ncols, i + 1)
        sp.axis('Off')  # Don't show axes (or gridlines)

        img = mpimg.imread(img_path)
        plt.imshow(img)

    plt.show()


Error: One or both of the directories do not exist.


## Building a Small Model from Scratch

But before we continue, let's start defining the model:

Step 1 will be to import tensorflow.

In [12]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [13]:
import tensorflow as tf
print(tf.__version__)

2.17.1


We then add convolutional layers as in the previous example, and flatten the final result to feed into the densely connected layers.

Finally we add the densely connected layers.

Note that because we are facing a two-class classification problem, i.e. a *binary classification problem*, we will end our network with a [*sigmoid* activation](https://wikipedia.org/wiki/Sigmoid_function), so that the output of our network will be a single scalar between 0 and 1, encoding the probability that the current image is class 1 (as opposed to class 0).

In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',
                           input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


The model.summary() method call prints a summary of the NN

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 298, 298, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 149, 149, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 147, 147, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 73, 73, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 71, 71, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 35, 35, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 78400)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      40,141,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,165,409 (153.22 MB)

 Trainable params: 40,165,409 (153.22 MB)

 Non-trainable params: 0 (0.00 B)

The "output shape" column shows how the size of your feature map evolves in each successive layer. The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the dimensions.

Next, we'll configure the specifications for model training. We will train our model with the `binary_crossentropy` loss, because it's a binary classification problem and our final activation is a sigmoid. (For a refresher on loss metrics, see the [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture).) We will use the `rmsprop` optimizer with a learning rate of `0.001`. During training, we will want to monitor classification accuracy.

**NOTE**: In this case, using the [RMSprop optimization algorithm](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) is preferable to [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), because RMSprop automates learning-rate tuning for us. (Other optimizers, such as [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) and [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), also automatically adapt the learning rate during training, and would work equally well here.)

In [17]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.001),  # Use learning_rate instead of lr
    metrics=['accuracy']
)


### Data Preprocessing

Let's set up data generators that will read pictures in our source folders, convert them to `float32` tensors, and feed them (with their labels) to our network. We'll have one generator for the training images and one for the validation images. Our generators will yield batches of images of size 300x300 and their labels (binary).

As you may already know, data that goes into neural networks should usually be normalized in some way to make it more amenable to processing by the network. (It is uncommon to feed raw pixels into a convnet.) In our case, we will preprocess our images by normalizing the pixel values to be in the `[0, 1]` range (originally all values are in the `[0, 255]` range).

In Keras this can be done via the `keras.preprocessing.image.ImageDataGenerator` class using the `rescale` parameter. This `ImageDataGenerator` class allows you to instantiate generators of augmented image batches (and their labels) via `.flow(data, labels)` or `.flow_from_directory(directory)`. These generators can then be used with the Keras model methods that accept data generators as inputs: `fit_generator`, `evaluate_generator`, and `predict_generator`.

In [18]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to the ZIP files and their extraction directories
horse_or_human_zip = '/tmp/horse-or-human.zip'
validation_horse_or_human_zip = '/tmp/validation-horse-or-human.zip'
train_dir = '/tmp/horse-or-human/'
validation_dir = '/tmp/validation-horse-or-human/'

# Ensure the ZIP files exist (or download if necessary)
if not os.path.exists(horse_or_human_zip) or not os.path.exists(validation_horse_or_human_zip):
    print("One or both ZIP files are missing. Please ensure the files are available.")
else:
    # Extract the ZIP files
    def extract_zip(file_path, extract_to):
        if os.path.exists(file_path):
            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)
                    print(f"Extracted {file_path} to {extract_to}")
            except zipfile.BadZipFile:
                print(f"Error: {file_path} is not a valid ZIP file.")
        else:
            print(f"Error: {file_path} does not exist.")

    extract_zip(horse_or_human_zip, train_dir)
    extract_zip(validation_horse_or_human_zip, validation_dir)

# Check if the directories exist after extraction
if not os.path.exists(train_dir) or not os.path.exists(validation_dir):
    print("Extraction failed. Ensure the ZIP files contain the correct directories.")
else:
    # Step 2: Create Data Generators
    # All images will be rescaled by 1./255
    train_datagen = ImageDataGenerator(rescale=1/255)
    validation_datagen = ImageDataGenerator(rescale=1/255)

    # Flow training images in batches using train_datagen generator
    train_generator = train_datagen.flow_from_directory(
            train_dir,                 # Source directory for training images
            target_size=(300, 300),    # Resize images to 300x300
            batch_size=128,
            class_mode='binary')       # Binary labels for binary_crossentropy

    # Flow validation images in batches using validation_datagen generator
    validation_generator = validation_datagen.flow_from_directory(
            validation_dir,            # Source directory for validation images
            target_size=(300, 300),    # Resize images to 300x300
            batch_size=32,
            class_mode='binary')       # Binary labels for binary_crossentropy



Error: /tmp/horse-or-human.zip is not a valid ZIP file.
Error: /tmp/validation-horse-or-human.zip is not a valid ZIP file.
Extraction failed. Ensure the ZIP files contain the correct directories.


### Training
Let's train for 15 epochs -- this may take a few minutes to run.

Do note the values per epoch.

The Loss and Accuracy are a great indication of progress of training. It's making a guess as to the classification of the training data, and then measuring it against the known label, calculating the result. Accuracy is the portion of correct guesses.

In [19]:
import os
import zipfile
import requests
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models

# Function to download and save a file
def download_file(url, destination):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(destination, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {destination}")
    else:
        print(f"Failed to download {url}")

# URLs for the dataset
horse_or_human_url = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
validation_horse_or_human_url = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"

# Paths to save the files in Colab's /content directory
horse_or_human_zip = '/content/horse-or-human.zip'
validation_horse_or_human_zip = '/content/validation-horse-or-human.zip'

# Download the datasets if not already present
if not os.path.exists(horse_or_human_zip):
    download_file(horse_or_human_url, horse_or_human_zip)
if not os.path.exists(validation_horse_or_human_zip):
    download_file(validation_horse_or_human_url, validation_horse_or_human_zip)

# Extract the ZIP files
train_dir = '/content/horse-or-human/'
validation_dir = '/content/validation-horse-or-human/'

def extract_zip(file_path, extract_to):
    if os.path.exists(file_path):
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
                print(f"Extracted {file_path} to {extract_to}")
        except zipfile.BadZipFile:
            print(f"Error: {file_path} is not a valid ZIP file.")
    else:
        print(f"Error: {file_path} does not exist.")

extract_zip(horse_or_human_zip, train_dir)
extract_zip(validation_horse_or_human_zip, validation_dir)

# Check if directories exist
if not os.path.exists(train_dir) or not os.path.exists(validation_dir):
    print("Dataset extraction failed. Ensure the downloaded files are valid.")
else:
    # Data generators
    train_datagen = ImageDataGenerator(rescale=1/255)
    validation_datagen = ImageDataGenerator(rescale=1/255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(300, 300),
        batch_size=128,
        class_mode='binary')

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')

    # Define the model
    model = models.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Binary classification output
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=15,
        steps_per_epoch=8,
        validation_steps=8,
        verbose=1)

    # Save the model (optional)
    model.save('/content/horse_or_human_model.h5')
    print("Model training complete and saved.")


Extracted /content/horse-or-human.zip to /content/horse-or-human/
Extracted /content/validation-horse-or-human.zip to /content/validation-horse-or-human/
Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 765ms/step - accuracy: 0.5161 - loss: 3.1061 - val_accuracy: 0.8086 - val_loss: 0.4621
Epoch 2/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8906 - loss: 0.4170 
Epoch 3/15


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 429ms/step - accuracy: 0.9039 - loss: 0.3152 - val_accuracy: 0.6445 - val_loss: 1.4451
Epoch 4/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.0650
Epoch 5/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 398ms/step - accuracy: 0.9569 - loss: 0.1019 - val_accuracy: 0.6758 - val_loss: 2.2601
Epoch 6/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9844 - loss: 0.0632  
Epoch 7/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 457ms/step - accuracy: 0.9725 - loss: 0.0608 - val_accuracy: 0.7305 - val_loss: 2.0970
Epoch 8/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0137  
Epoch 9/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 406ms/step - accuracy: 0.9974 - loss: 0.0186 - val_accuracy: 0.7617 - val_loss: 1.9858
Epoch 10/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 1.0000 - loss: 0.0056
Epoch 11/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 308ms/step - accuracy: 1.0000 - loss: 0.0066 - val_accuracy: 0.8047 - val_loss: 1.7320
Epoch 12/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

Model training complete and saved.


###Running the Model

Let's now take a look at actually running a prediction using the model. This code will allow you to choose 1 or more files from your file system, it will then upload them, and run them through the model, giving an indication of whether the object is a horse or a human.

You can download images from the internet to your file system to try them out!

Note that you might see that the network makes a LOT of mistakes, despite the fact that the training accuracy is above 99%.

This is due to something called **overfitting**, which means that the neural network is trained with very limited data -- there are only 500ish images of each class. So it's very good at recognizing images that look like those in the training set, but it can fail a lot at images that are not in the training set.

This is a data point proving that the more data you train on, the better your final network will be!

There are many techniques that can be used to make your training better, despite limited data, including something called Image Augmentation. That's beyond the scope of this lab!

In [20]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = x / 255
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")


Saving 9e07a57ff3be7a82_856.jpeg to 9e07a57ff3be7a82_856 (1).jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 701ms/step
[0.983493]
9e07a57ff3be7a82_856 (1).jpeg is a human


### Visualizing Intermediate Representations

To get a feel for what kind of features our convnet has learned, one fun thing to do is to visualize how an input gets transformed as it goes through the convnet.

Let's pick a random image from the training set, and then generate a figure where each row is the output of a layer, and each image in the row is a specific filter in that output feature map. Rerun this cell to generate intermediate representations for a variety of training images.

In [3]:
import os
import zipfile
import requests
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow as tf # import tensorflow
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt

# ... (Rest of your code for downloading and extracting the dataset) ...

# Define the model
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification output
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define train_dir and validation_dir here (assuming they are defined in your data extraction part)
train_dir = '/content/horse-or-human/'  # Replace with your actual path
validation_dir = '/content/validation-horse-or-human/'  # Replace with your actual path

# Data generators
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=128,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='binary')

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,
    steps_per_epoch=8,
    validation_steps=8,
    verbose=1)

# Call the model once to create the input attribute
# You can use a dummy input of the correct shape
dummy_input = np.zeros((1, 300, 300, 3)) # Example dummy input
_ = model.predict(dummy_input)

# Now you can create the visualization model
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)


# ... (Rest of your code for visualization) ...

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 486ms/step - accuracy: 0.5162 - loss: 2.8607 - val_accuracy: 0.8398 - val_loss: 0.4634
Epoch 2/15
1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.9219 - loss: 0.5270

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - accuracy: 0.9219 - loss: 0.5270
Epoch 3/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 393ms/step - accuracy: 0.9134 - loss: 0.3903 - val_accuracy: 0.8125 - val_loss: 0.7316
Epoch 4/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8906 - loss: 0.2208  
Epoch 5/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 396ms/step - accuracy: 0.9082 - loss: 0.2221 - val_accuracy: 0.8047 - val_loss: 1.0726
Epoch 6/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9844 - loss: 0.1070  
Epoch 7/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 308ms/step - accuracy: 0.9522 - loss: 0.1243 - val_accuracy: 0.7891 - val_loss: 1.5905
Epoch 8/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9844 - loss: 0.0466  
Epoch 9/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 466ms/step - accuracy: 0.9804 - loss: 0.0535 - val_accuracy: 0.8047 - val_loss: 1.9456
Epoch 10/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 1.0000 - loss: 0.0160
Epoch 11/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 420ms/step - accuracy: 0.9908 - loss: 0.031

ValueError: The layer sequential_2 has never been called and thus has no defined input.

As you can see we go from the raw pixels of the images to increasingly abstract and compact representations. The representations downstream start highlighting what the network pays attention to, and they show fewer and fewer features being "activated"; most are set to zero. This is called "sparsity." Representation sparsity is a key feature of deep learning.


These representations carry increasingly less information about the original pixels of the image, but increasingly refined information about the class of the image. You can think of a convnet (or a deep network in general) as an information distillation pipeline.

## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)